In [1]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm.auto import tqdm
from scipy.io import loadmat

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, Subset, ConcatDataset
from pipeline_improved import VAE1D, beta_cyclic, beta_linear, compute_scores, XGBClassifier
from torch.utils.data import DataLoader, TensorDataset, Subset, ConcatDataset

In [54]:
import os
from preprocess import load_all_anomalies, compute_zscore_stats, apply_zscore, load_sanos


def find_data_subfolder(subfolder_name, start_path='.'):
    current_path = os.path.abspath(start_path)
    while True:
        candidate = os.path.join(current_path, 'data', subfolder_name)
        if os.path.isdir(candidate):
            return candidate
        parent = os.path.dirname(current_path)
        if parent == current_path:
            break
        current_path = parent
    return None

def find_file(root, filename):
    """
    Busca recursivamente `filename` bajo `root` y devuelve la ruta completa.
    """
    for r, _, files in os.walk(root):
        if filename in files:
            return os.path.join(r, filename)
    return None


# Ahora buscás las rutas relativas automáticamente:
PTB_DIR = find_data_subfolder('ptb-xl/1.0.3')
CHAP_DIR = find_data_subfolder('ChapmanShaoxing')
MIT_DIR = find_data_subfolder('mitdb')

In [ ]:
normals = load_sanos(PTB_DIR, CHAP_DIR)     
ptb_df  = pd.read_csv(find_file(PTB_DIR, 'ptbxl_database.csv')) 

In [ ]:
anomalies = load_all_anomalies(PTB_DIR, CHAP_DIR, ptb_df)  # (N_ano,1,L) 

In [37]:
# 2) Split normales en DEV y TEST (80/20)
n_norm = normals.shape[0]
split_dev = int(0.8 * n_norm)
dev_norm  = normals[:split_dev]
test_norm = normals[split_dev:]

# 3) Dentro de DEV, split en TRAIN y VAL (80/20 of DEV)
n_dev = dev_norm.shape[0]
split_train = int(0.8 * n_dev)
train_norm = dev_norm[:split_train]
val_norm   = dev_norm[split_train:]

# 4) Normalización Z-score usando solo train_norm) Normalización Z-score usando solo train_norm
mean, std = compute_zscore_stats(train_norm)
train_norm = apply_zscore(train_norm, mean, std)
val_norm   = apply_zscore(val_norm,   mean, std)
test_norm  = apply_zscore(test_norm,  mean, std)
anomalies  = apply_zscore(anomalies,  mean, std)

# 4) Convertir a tensores
train_tensor = torch.tensor(train_norm, dtype=torch.float32)
val_tensor   = torch.tensor(val_norm,   dtype=torch.float32)
test_tensor  = torch.tensor(test_norm,  dtype=torch.float32)
ano_tensor   = torch.tensor(anomalies,  dtype=torch.float32)


In [38]:
# Parámetros de entrenamiento final
latent_dim = 32
lr         = 1e-3
n_blocks   = 3
epochs     = 50
batch_size = 32


In [25]:

# 5) Entrena VAE sobre conjunto DEV (train_norm + val_norm)
dev_tensor = torch.cat([train_tensor, val_tensor], dim=0)
dev_ds     = TensorDataset(dev_tensor, torch.zeros(len(dev_tensor)))
dev_loader = DataLoader(dev_ds, batch_size=batch_size, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE1D(input_ch=1, latent_dim=latent_dim, n_blocks=n_blocks).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for x, _ in dev_loader:
        x = x.to(device)
        mu, logv = model.encode(x)
        z = model.reparameterize(mu, logv)
        rec = model.decode(z)
        recon_loss = ((rec - x)**2).mean()
        kl_loss = (-0.5 * (1 + logv - mu.pow(2) - logv.exp()).sum()) / x.size(0)
        beta = beta_cyclic(epoch, cycle=10, beta_max=4.0)
        loss = recon_loss + beta * kl_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dev_loader):.4f}")


Epoch 1/50, Loss: 1.2389
Epoch 2/50, Loss: 534.0065
Epoch 3/50, Loss: 39.3720
Epoch 4/50, Loss: 31.5000
Epoch 5/50, Loss: 35.4579
Epoch 6/50, Loss: 25.8717
Epoch 7/50, Loss: 21.8407
Epoch 8/50, Loss: 18.6034
Epoch 9/50, Loss: 18.6262
Epoch 10/50, Loss: 17.6717
Epoch 11/50, Loss: 1.0440
Epoch 12/50, Loss: 2.5172
Epoch 13/50, Loss: 2.0412
Epoch 14/50, Loss: 2.1568
Epoch 15/50, Loss: 2.3338
Epoch 16/50, Loss: 3.1153
Epoch 17/50, Loss: 7.5371
Epoch 18/50, Loss: 12.1489
Epoch 19/50, Loss: 12.9659
Epoch 20/50, Loss: 14.2619
Epoch 21/50, Loss: 0.9956
Epoch 22/50, Loss: 1.7647
Epoch 23/50, Loss: 1.6853
Epoch 24/50, Loss: 1.4679
Epoch 25/50, Loss: 1.4739
Epoch 26/50, Loss: 1.5725
Epoch 27/50, Loss: 1.7373
Epoch 28/50, Loss: 2.6681
Epoch 29/50, Loss: 5.3447
Epoch 30/50, Loss: 10.8783
Epoch 31/50, Loss: 0.9921
Epoch 32/50, Loss: 1.7011
Epoch 33/50, Loss: 1.5005
Epoch 34/50, Loss: 1.2631
Epoch 35/50, Loss: 1.1929
Epoch 36/50, Loss: 1.1977
Epoch 37/50, Loss: 1.2838
Epoch 38/50, Loss: 1.6771
Epoch 3

In [31]:

# 6) Prepara test final balanceado: test_norm vs primeras anomalías: test_norm vs primeras anomalías
N_test = test_tensor.shape[0]
test_x = torch.cat([test_tensor, ano_tensor[:N_test]], dim=0)
y_test = np.concatenate([np.zeros(N_test), np.ones(N_test)])
test_loader = DataLoader(TensorDataset(test_x, torch.tensor(y_test, dtype=torch.long)), batch_size=batch_size)


In [32]:
# --- 7) Obtiene scores en DEV y TEST por separado ---
beta_last = beta_cyclic(epochs-1, cycle=10, beta_max=4.0)

# 7a) DEV scores y latentes
# -------------------------
# Crea DEV loader (train_norm + val_norm)
dev_tensor = torch.cat([train_tensor, val_tensor], dim=0)
dev_labels = np.concatenate([
    np.zeros(len(train_tensor)),        # sanos
    np.zeros(len(val_tensor))          # sanos (aquí no hay anomalías)
])
# (si quieres incluir anomalías de DEV, tendrías que extraer un subset de anomalies)
# Para un detector puramente no supervisado podrías omitir el XGBoost en DEV.



In [ ]:

dev_loader = DataLoader(TensorDataset(dev_tensor, torch.tensor(dev_labels)), batch_size=batch_size)

errs_dev, zs_dev = compute_scores(model, dev_loader, device, beta_last)
# Limpia de inf/nan
errs_dev = np.nan_to_num(errs_dev, nan=0.0, posinf=1e6, neginf=-1e6)
zs_dev   = np.nan_to_num(zs_dev,   nan=0.0, posinf=1e6, neginf=-1e6)

X_dev = np.hstack([errs_dev.reshape(-1,1), zs_dev])
y_dev = dev_labels  # etiquetas de DEV

# Entrena XGBoost en DEV
clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf.fit(X_dev, y_dev)


In [ ]:

# --- 7b) TEST scores y latentes ---
# -------------------------
errs_test, zs_test = compute_scores(model, test_loader, device, beta_last)
errs_test = np.nan_to_num(errs_test, nan=0.0, posinf=1e6, neginf=-1e6)
zs_test   = np.nan_to_num(zs_test,   nan=0.0, posinf=1e6, neginf=-1e6)

X_test = np.hstack([errs_test.reshape(-1,1), zs_test])
y_test  = np.concatenate([np.zeros(len(test_tensor)), np.ones(len(ano_tensor[:len(test_tensor)]))])

# Solo inferencia sobre TEST
probs = clf.predict_proba(X_test)[:,1]


In [29]:

from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, r2_score


In [ ]:

# --- 10) Métricas finales en TEST ---
auc      = roc_auc_score(y_test, probs)
y_pred   = (probs >= 0.5).astype(int)
metrics  = {
    'roc_auc':   auc,
    'precision': precision_score(y_test, y_pred),
    'recall':    recall_score(y_test, y_pred),
    'f1':        f1_score(y_test, y_pred),
    'accuracy':  accuracy_score(y_test, y_pred),
    'r2':        r2_score(y_test, probs)
}
print('Final metrics on TEST (unseen):', metrics)

Final metrics: {'roc_auc': np.float64(1.0), 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'accuracy': 1.0, 'r2': 0.9999610804432385}


In [39]:

# 5) Entrena VAE sobre conjunto DEV (train_norm + val_norm)
dev_tensor = torch.cat([train_tensor, val_tensor], dim=0)
dev_ds     = TensorDataset(dev_tensor, torch.zeros(len(dev_tensor)))
dev_loader = DataLoader(dev_ds, batch_size=batch_size, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE1D(input_ch=1, latent_dim=latent_dim, n_blocks=n_blocks).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for x, _ in dev_loader:
        x = x.to(device)
        mu, logv = model.encode(x)
        z = model.reparameterize(mu, logv)
        rec = model.decode(z)
        recon_loss = ((rec - x)**2).mean()
        kl_loss = (-0.5 * (1 + logv - mu.pow(2) - logv.exp()).sum()) / x.size(0)
        beta = beta_cyclic(epoch, cycle=10, beta_max=4.0)
        loss = recon_loss + beta * kl_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dev_loader):.4f}")


Epoch 1/50, Loss: 1.1958
Epoch 2/50, Loss: 202.6760
Epoch 3/50, Loss: 16.5132
Epoch 4/50, Loss: 12.4264
Epoch 5/50, Loss: 9.1345
Epoch 6/50, Loss: 8.9151
Epoch 7/50, Loss: 8.4336
Epoch 8/50, Loss: 7.5735
Epoch 9/50, Loss: 7.4693
Epoch 10/50, Loss: 7.1344
Epoch 11/50, Loss: 1.0399
Epoch 12/50, Loss: 2.1132
Epoch 13/50, Loss: 1.5026
Epoch 14/50, Loss: 1.3857
Epoch 15/50, Loss: 1.4055
Epoch 16/50, Loss: 1.5078
Epoch 17/50, Loss: 2.4385
Epoch 18/50, Loss: 4.3530
Epoch 19/50, Loss: 9.3779
Epoch 20/50, Loss: 7.1084
Epoch 21/50, Loss: 0.9955
Epoch 22/50, Loss: 1.3916
Epoch 23/50, Loss: 1.3126
Epoch 24/50, Loss: 1.2037
Epoch 25/50, Loss: 1.1607
Epoch 26/50, Loss: 1.1495
Epoch 27/50, Loss: 1.1525
Epoch 28/50, Loss: 1.2128
Epoch 29/50, Loss: 1.4502
Epoch 30/50, Loss: 2.4173
Epoch 31/50, Loss: 0.9905
Epoch 32/50, Loss: 1.2744
Epoch 33/50, Loss: 1.1224
Epoch 34/50, Loss: 1.0621
Epoch 35/50, Loss: 1.0562
Epoch 36/50, Loss: 1.0880
Epoch 37/50, Loss: 1.4443
Epoch 38/50, Loss: 2.8737
Epoch 39/50, Loss

In [40]:

# 6) Split anomalies into DEV and TEST (80/20)
n_ano = anomalies.shape[0]
split_ano = int(0.8 * n_ano)
ano_dev  = anomalies[:split_ano]
ano_test = anomalies[split_ano:]


In [52]:

import numpy as np
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score,
    f1_score, accuracy_score, r2_score, roc_curve
)
from sklearn.covariance import EmpiricalCovariance
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

# --- 7) Obtener scores ELBO y latentes en TEST final ---
beta_last = beta_cyclic(epochs-1, cycle=10, beta_max=4.0)
errs_test, zs_test = compute_scores(model, test_loader, device, beta_last)

# Limpiar inf/nan
maxv = np.finfo(np.float32).max/10
minv = np.finfo(np.float32).min/10
errs_test = np.nan_to_num(errs_test, nan=0.0, posinf=maxv, neginf=minv)
zs_test   = np.nan_to_num(zs_test,   nan=0.0, posinf=maxv, neginf=minv)

# Etiquetas reales de TEST
# test_norm y ano_test definidos previamente
N_test = test_norm.shape[0]
y_test = np.concatenate([np.zeros(N_test), np.ones(N_test)])

# --- 8) Mahalanobis en espacio latente ---
# Ajustamos la covarianza solo con z de normales de DEV (train+val)
# dev_normals: numpy array (N_train+N_val,1,L)
dev_normals = np.concatenate([train_norm, val_norm], axis=0)

# Para obtener z_train_norm
errs_dev_norm, zs_dev_norm = compute_scores(
    model,
    DataLoader(
        TensorDataset(
            torch.tensor(dev_normals, dtype=torch.float32),
            torch.zeros(len(dev_normals))  # labels zeros, no importan aquí
        ),
        batch_size=batch_size,
        shuffle=False
    ),
    device,
    beta_last
)

zs_dev_norm = np.nan_to_num(zs_dev_norm, nan=0.0, posinf=maxv, neginf=minv)
cov_est = EmpiricalCovariance().fit(zs_dev_norm)
# distancia Mahalanobis para TEST
dM = cov_est.mahalanobis(zs_test)
dM = np.sqrt(dM)

# --- 9) Detectores no supervisados y thresholding ---

results = {}

# 9a) ELBO-threshold
probs_elbo = -errs_test
fpr, tpr, thr = roc_curve(y_test, probs_elbo)
youden = np.argmax(tpr - fpr)
thr_elbo = thr[youden]
y_pred_elbo = (probs_elbo >= thr_elbo).astype(int)


results['ELBO'] = {
    'auc': roc_auc_score(y_test, probs_elbo),
    'precision': precision_score(y_test, y_pred_elbo),
    'recall':    recall_score(y_test, y_pred_elbo),
    'f1':        f1_score(y_test, y_pred_elbo),
    'accuracy':  accuracy_score(y_test, y_pred_elbo),
    'r2':        r2_score(y_test, probs_elbo)
}

# 9b) Mahalanobis-threshold
probs_maha = dM
fpr2, tpr2, thr2 = roc_curve(y_test, probs_maha)
youden2 = np.argmax(tpr2 - fpr2)
thr_maha = thr2[youden2]
y_pred_maha = (probs_maha >= thr_maha).astype(int)
results['Mahalanobis'] = {
    'auc': roc_auc_score(y_test, probs_maha),
    'precision': precision_score(y_test, y_pred_maha),
    'recall':    recall_score(y_test, y_pred_maha),
    'f1':        f1_score(y_test, y_pred_maha),
    'accuracy':  accuracy_score(y_test, y_pred_maha),
    'r2':        r2_score(y_test, probs_maha)
}

# 9c) Isolation Forest sobre z
iso = IsolationForest(contamination=N_test/len(zs_dev_norm), random_state=0)
iso.fit(zs_dev_norm)
scores_iso = -iso.decision_function(zs_test)
fpr3, tpr3, thr3 = roc_curve(y_test, scores_iso)
youden3 = np.argmax(tpr3 - fpr3)
thr_iso = thr3[youden3]
y_pred_iso = (scores_iso >= thr_iso).astype(int)
results['IsolationForest'] = {
    'auc': roc_auc_score(y_test, scores_iso),
    'precision': precision_score(y_test, y_pred_iso),
    'recall':    recall_score(y_test, y_pred_iso),
    'f1':        f1_score(y_test, y_pred_iso),
    'accuracy':  accuracy_score(y_test, y_pred_iso),
    'r2':        r2_score(y_test, scores_iso)
}

# 9d) One-Class SVM sobre z
ocsvm = OneClassSVM(gamma='auto', nu=0.05)
ocsvm.fit(zs_dev_norm)
scores_oc = -ocsvm.decision_function(zs_test)
fpr4, tpr4, thr4 = roc_curve(y_test, scores_oc)
youden4 = np.argmax(tpr4 - fpr4)
thr_oc = thr4[youden4]
y_pred_oc = (scores_oc >= thr_oc).astype(int)
results['OC-SVM'] = {
    'auc': roc_auc_score(y_test, scores_oc),
    'precision': precision_score(y_test, y_pred_oc),
    'recall':    recall_score(y_test, y_pred_oc),
    'f1':        f1_score(y_test, y_pred_oc),
    'accuracy':  accuracy_score(y_test, y_pred_oc),
    'r2':        r2_score(y_test, scores_oc)
}

# --- 10) Mostrar todos los resultados ---
for name, m in results.items():
    print(f"\n=== {name} ===")
    print(f"AUC      : {m['auc']:.3f}")
    print(f"Precision: {m['precision']:.3f}")
    print(f"Recall   : {m['recall']:.3f}")
    print(f"F1       : {m['f1']:.3f}")
    print(f"Accuracy : {m['accuracy']:.3f}")
    print(f"R2       : {m['r2']:.3f}")


=== ELBO ===
AUC      : 0.548
Precision: 0.637
Recall   : 0.278
F1       : 0.387
Accuracy : 0.560
R2       : -98.093

=== Mahalanobis ===
AUC      : 0.586
Precision: 0.614
Recall   : 0.366
F1       : 0.458
Accuracy : 0.568
R2       : -147.233

=== IsolationForest ===
AUC      : 0.562
Precision: 0.624
Recall   : 0.279
F1       : 0.386
Accuracy : 0.555
R2       : -1.000

=== OC-SVM ===
AUC      : 0.561
Precision: 0.629
Recall   : 0.265
F1       : 0.373
Accuracy : 0.554
R2       : -1181.057


In [51]:
# 2) Split normales en DEV y TEST (80/20)
n_norm = normals.shape[0]
split_dev = int(0.8 * n_norm)
dev_norm  = normals[:split_dev]
test_norm = normals[split_dev:]

# 3) Dentro de DEV, split en TRAIN y VAL (80/20 of DEV)
n_dev = dev_norm.shape[0]
split_train = int(0.8 * n_dev)
train_norm = dev_norm[:split_train]
val_norm   = dev_norm[split_train:]

# 4) Normalización Z-score usando solo train_norm) Normalización Z-score usando solo train_norm
mean, std = compute_zscore_stats(train_norm)
train_norm = apply_zscore(train_norm, mean, std)
val_norm   = apply_zscore(val_norm,   mean, std)
test_norm  = apply_zscore(test_norm,  mean, std)
anomalies  = apply_zscore(anomalies,  mean, std)

# 4) Convertir a tensores
train_tensor = torch.tensor(train_norm, dtype=torch.float32)
val_tensor   = torch.tensor(val_norm,   dtype=torch.float32)
test_tensor  = torch.tensor(test_norm,  dtype=torch.float32)
ano_tensor   = torch.tensor(anomalies,  dtype=torch.float32)

# Parámetros de entrenamiento final
latent_dim = 32
lr         = 1e-3
n_blocks   = 3
epochs     = 50
batch_size = 32

# 5) Entrena VAE sobre conjunto DEV (train_norm + val_norm)
dev_tensor = torch.cat([train_tensor, val_tensor], dim=0)
dev_ds     = TensorDataset(dev_tensor, torch.zeros(len(dev_tensor)))
dev_loader = DataLoader(dev_ds, batch_size=batch_size, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE1D(input_ch=1, latent_dim=latent_dim, n_blocks=n_blocks).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for x, _ in dev_loader:
        x = x.to(device)
        mu, logv = model.encode(x)
        z = model.reparameterize(mu, logv)
        rec = model.decode(z)
        recon_loss = ((rec - x)**2).mean()
        kl_loss = (-0.5 * (1 + logv - mu.pow(2) - logv.exp()).sum()) / x.size(0)
        beta = beta_cyclic(epoch, cycle=10, beta_max=4.0)
        loss = recon_loss + beta * kl_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dev_loader):.4f}")

    # 6) Split anomalies into DEV and TEST (80/20)
n_ano = anomalies.shape[0]
split_ano = int(0.8 * n_ano)
ano_dev  = anomalies[:split_ano]
ano_test = anomalies[split_ano:]

# 7) Entrena XGBoost y evalúa:
#   a) Genera scores ELBO y z para DEV (normales DEV + anomalías DEV)
# DEV normales ya en dev_tensor (train+val), dev_normals_labels = zeros
dev_normals = np.concatenate([train_norm, val_norm], axis=0)
dev_labels_norm = np.zeros(dev_normals.shape[0])
# DEV anomalies
dev_labels_ano = np.ones(ano_dev.shape[0])
# Prepara loader para DEV classifier scoring
dev_x = np.concatenate([dev_normals, ano_dev], axis=0)
dev_y = np.concatenate([dev_labels_norm, dev_labels_ano])
# TensorDataset expects torch tensors
dev_loader = DataLoader(
    TensorDataset(
        torch.tensor(dev_x, dtype=torch.float32),
        torch.tensor(dev_y, dtype=torch.long)
    ), batch_size=batch_size, shuffle=False
)
errs_dev, zs_dev = compute_scores(model, dev_loader, device, beta_last)
errs_dev = np.nan_to_num(errs_dev, nan=0.0, posinf=1e6, neginf=-1e6)
zs_dev   = np.nan_to_num(zs_dev,   nan=0.0, posinf=1e6, neginf=-1e6)
X_dev = np.hstack([errs_dev.reshape(-1,1), zs_dev])
y_dev = dev_y

# Entrena XGBoost en DEV (ahora con dos clases)
clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf.fit(X_dev, y_dev)

# 8) Evalúa en TEST final (normales TEST + anomalías TEST)
N_test = test_norm.shape[0]
test_x = np.concatenate([test_norm, ano_test[:N_test]], axis=0)
test_y = np.concatenate([np.zeros(N_test), np.ones(N_test)])
# Scores TEST
test_loader = DataLoader(
    TensorDataset(
        torch.tensor(test_x, dtype=torch.float32),
        torch.tensor(test_y, dtype=torch.long)
    ), batch_size=batch_size, shuffle=False
)
errs_test, zs_test = compute_scores(model, test_loader, device, beta_last)
errs_test = np.nan_to_num(errs_test, nan=0.0, posinf=1e6, neginf=-1e6)
zs_test   = np.nan_to_num(zs_test,   nan=0.0, posinf=1e6, neginf=-1e6)
X_test = np.hstack([errs_test.reshape(-1,1), zs_test])
y_test = test_y
# Predicciones
probs = clf.predict_proba(X_test)[:,1]

# 9) Métricas finales en TEST
auc      = roc_auc_score(y_test, probs)
y_pred   = (probs >= 0.5).astype(int)
metrics  = {
    'roc_auc':   auc,
    'precision': precision_score(y_test, y_pred),
    'recall':    recall_score(y_test, y_pred),
    'f1':        f1_score(y_test, y_pred),
    'accuracy':  accuracy_score(y_test, y_pred),
    'r2':        r2_score(y_test, probs)
}
print('Final metrics on unseen TEST:', metrics)


Epoch 1/50, Loss: 1.2378
Epoch 2/50, Loss: 1146.0367
Epoch 3/50, Loss: 43.5596
Epoch 4/50, Loss: 29.1880
Epoch 5/50, Loss: 34.6443
Epoch 6/50, Loss: 31.6987
Epoch 7/50, Loss: 26.9225
Epoch 8/50, Loss: 22.3716
Epoch 9/50, Loss: 19.0170
Epoch 10/50, Loss: 20.0865
Epoch 11/50, Loss: 1.1884
Epoch 12/50, Loss: 3.2446
Epoch 13/50, Loss: 2.7533
Epoch 14/50, Loss: 3.0681
Epoch 15/50, Loss: 3.4334
Epoch 16/50, Loss: 4.3026
Epoch 17/50, Loss: 5.7991
Epoch 18/50, Loss: 17.3754
Epoch 19/50, Loss: 18.6135
Epoch 20/50, Loss: 18.5862
Epoch 21/50, Loss: 1.0847
Epoch 22/50, Loss: 2.7654
Epoch 23/50, Loss: 2.2581
Epoch 24/50, Loss: 1.9074
Epoch 25/50, Loss: 1.8397
Epoch 26/50, Loss: 1.9659
Epoch 27/50, Loss: 2.1030
Epoch 28/50, Loss: 3.8655
Epoch 29/50, Loss: 8.3550
Epoch 30/50, Loss: 13.0368
Epoch 31/50, Loss: 1.0300
Epoch 32/50, Loss: 2.1466
Epoch 33/50, Loss: 1.7997
Epoch 34/50, Loss: 1.4958
Epoch 35/50, Loss: 1.4171
Epoch 36/50, Loss: 1.5791
Epoch 37/50, Loss: 3.4036
Epoch 38/50, Loss: 8.4945
Epoch 